In [4]:
import os
# Change the working directory to strategy_lab
os.chdir("/Users/sugang/Desktop/projects/2024coin/strategy_lab")


import pandas as pd
import numpy as np
from handle_candle import resample_df
from datetime import datetime
from performance_utils import get_performance


In [5]:
def backtest(df, short_ma=10, long_ma=20):
    df = df.copy()
    # Calculate MACD components
    df["short_volume_ma"] = df["volume_krw"].rolling(short_ma).mean()
    df["long_volume_ma"] = df["volume_krw"].rolling(long_ma).mean()
    
    # Initialize the signal column with default value 0
    df["signal"] = 0
    
    # Generate buy signal: 1 when MACD crosses above zero
    df.loc[(df["short_volume_ma"] > df["long_volume_ma"]) & (df["short_volume_ma"].shift(1).fillna(0) <= df["long_volume_ma"].shift(1).fillna(0)), "signal"] = 1
    
    # Generate sell signal: -1 when MACD crosses below zero
    df.loc[(df["short_volume_ma"] < df["long_volume_ma"]) & (df["short_volume_ma"].shift(1).fillna(0) >= df["long_volume_ma"].shift(1).fillna(0)), "signal"] = -1
    
    # Initialize the position column with default value 0
    df["position"] = 0
    
    # Variable to track the current position status
    current_position = 0
    
    # Iterate over the rows and set the position
    for i in range(1, len(df)):
        if df.loc[i-1, "signal"] == 1:  # Enter long position
            if df.loc[i, "signal"] == -1:
                df.loc[i, "position"] = 1
                current_position = 0
                continue
            current_position = 1
        elif df.loc[i, "signal"] == -1:  # Exit position
            df.loc[i, "position"] = current_position
            current_position = 0
            continue
        df.loc[i, "position"] = current_position
    
    # Calculate the strategy returns (only when in a long position)
    df["strategy_returns"] = df["position"] * df["close"].pct_change()
    df["strategy_returns2"] = df["strategy_returns"]
    
    # Adjust for trading fees (buy with 0.2% fee, sell with 0.2% fee)
    df["buy_price"] = df["close"].shift(1) * np.where(df["signal"].shift(1) == 1, 1.002, 1)
    df["sell_price"] = df["close"] * np.where(df["signal"] == -1, 0.998, 1)
    
    # Calculate strategy returns with fees
    df["strategy_returns2"] = np.where(df["position"] == 1, df["sell_price"] / df["buy_price"] - 1, 0)
    
    # Calculate the cumulative returns
    df["cumulative_returns"] = (1 + df["strategy_returns"]).cumprod()
    df["cumulative_returns2"] = (1 + df["strategy_returns2"]).cumprod()
    performance = get_performance(df)
    return {"strategy": f"obv_{short_ma}_{long_ma}", "stoploss": None, **performance}

def backtest_stoploss(df, short_ma=10, long_ma=20, stop_loss_pct=0.05):
    df = df.copy()
    df["short_volume_ma"] = df["volume_krw"].rolling(short_ma).mean()
    df["long_volume_ma"] = df["volume_krw"].rolling(long_ma).mean()
    
    # Initialize the signal column with default value 0
    df["signal"] = 0
    
    # Generate buy signal: 1 when MACD crosses above zero
    df.loc[(df["short_volume_ma"] > df["long_volume_ma"]) & (df["short_volume_ma"].shift(1).fillna(0) <= df["long_volume_ma"].shift(1).fillna(0)), "signal"] = 1
    
    # Generate sell signal: -1 when MACD crosses below zero
    df.loc[(df["short_volume_ma"] < df["long_volume_ma"]) & (df["short_volume_ma"].shift(1).fillna(0) >= df["long_volume_ma"].shift(1).fillna(0)), "signal"] = -1
    
    # Initialize the position column with default value 0
    df["position"] = 0
    df["highest_price"] = np.nan
    
    # Variable to track the current position status
    current_position = 0
    
    # Iterate over the rows and set the position
    for i in range(1, len(df)):
        if df.loc[i-1, "signal"] == 1:  # Enter long position
            df.loc[i, "highest_price"] = max(df.loc[i-1, "close"], df.loc[i, "close"])
            if df.loc[i, "signal"] == -1:
                df.loc[i, "position"] = 1
                current_position = 0
                continue
                
            if df.loc[i, "close"] <= df.loc[i, "highest_price"] * (1 - stop_loss_pct):
                df.loc[i, "position"] = 1
                df.loc[i, "signal"] = -1
                current_position = 0
                continue
            current_position = 1
        elif df.loc[i, "signal"] == -1 and current_position == 1:  # Exit position
            df.loc[i, "highest_price"] = max(df.loc[i-1, "highest_price"], df.loc[i, "close"])
            df.loc[i, "position"] = current_position
            current_position = 0
            continue
        
        elif current_position == 1:  # Check current_position instead of df position
            df.loc[i, "highest_price"] = max(df.loc[i-1, "highest_price"], df.loc[i, "close"])
            if df.loc[i, "close"] <= df.loc[i, "highest_price"] * (1 - stop_loss_pct):
                df.loc[i, "position"] = 1
                df.loc[i, "signal"] = -1
                current_position = 0
                continue
        
        if current_position == 0 and df.loc[i, "short_volume_ma"] > df.loc[i, "long_volume_ma"]:
            df.loc[i, "signal"] = 1
        df.loc[i, "position"] = current_position
    
    # Calculate the strategy returns (only when in a long position)
    df["strategy_returns"] = df["position"] * df["close"].pct_change()
    df["strategy_returns2"] = df["strategy_returns"]
    
    # Adjust for trading fees (buy with 0.2% fee, sell with 0.2% fee)
    df["buy_price"] = df["close"].shift(1) * np.where(df["signal"].shift(1) == 1, 1.002, 1)
    df["sell_price"] = df["close"] * np.where(df["signal"] == -1, 0.998, 1)
    
    # Calculate strategy returns with fees
    df["strategy_returns2"] = np.where(df["position"] == 1, df["sell_price"] / df["buy_price"] - 1, 0)
    
    # Calculate the cumulative returns
    df["cumulative_returns"] = (1 + df["strategy_returns"]).cumprod()
    df["cumulative_returns2"] = (1 + df["strategy_returns2"]).cumprod()
    performance = get_performance(df)
    return {"strategy": f"obv_{short_ma}_{long_ma}", "stoploss": stop_loss_pct, **performance}

def backtest_benchmark(df):
    df = df.copy()
    df["strategy_returns2"] = df["close"].pct_change()
    df["cumulative_returns2"] = (1 + df["strategy_returns2"]).cumprod()
    performance = get_performance(df)
    # df.to_csv(f"3_moving_average_convergence_divergence/temp_benchmark.csv")
    return {"strategy": "benchmark", "stoploss": None, **performance}


In [6]:
# Parameters
short_periods = [3, 5, 8, 12, 15]
long_periods = [21, 26, 30, 50, 100]
stop_loss_pcts = [0.03, 0.05, 0.1, 0.2]
hours = range(0, 24, 1)

# Store results in a list
results = []
# Read the data
df = pd.read_csv("temp.csv", index_col=0)
execution_time = datetime.strptime("00:00", "%H:%M")
df = resample_df(df, execution_time=execution_time)
# df_copy = df.copy()
# execution_time = datetime.strptime("00:00", "%H:%M")
# df_copy = resample_df(df_copy, execution_time=execution_time)
# print(backtest(df_copy))

# Run backtests and collect results
results.append(backtest_benchmark(df))

df = pd.read_csv("temp.csv", index_col=0)
for hour in hours:
    df_copy = df.copy()
    execution_time = datetime.strptime(f"{hour:02d}:00", "%H:%M")
    df_copy = resample_df(df_copy, execution_time=execution_time)
    for short_period in short_periods:
        for long_period in long_periods:
            results.append({
                "execution_time": execution_time.strftime("%H:%M"),
                **backtest(df_copy, short_period, long_period)
            })
            for stop_loss_pct in stop_loss_pcts:
                results.append({
                    "execution_time": execution_time.strftime("%H:%M"),
                    **backtest_stoploss(df_copy, short_period, long_period, stop_loss_pct)
                })

# Save results
results_df = pd.DataFrame(results)
results_df.to_csv("5_on_balance_volume/results.csv", index=False)
print("Results saved to results.csv")


Results saved to results.csv
